<a href="https://colab.research.google.com/github/Sibaso/Project2_quan.nt173312/blob/master/Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Network
Lấy dữ liệu ở file ở dạng ma trận thưa
Chuyển dữ liệu sang kiểu tensor

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import torch

def load_data(data_path, vocab_size):
  with open(data_path, encoding = 'latin1') as f:
    d_lines = f.read().splitlines()
  data,labels = [],[]
  for data_id, line in enumerate(d_lines):
    vector = [0.0 for _ in range(vocab_size)]
    features = line.split('<fff>')
    label, doc_id = int(features[0]), int(features[1])
    for token in features[2].split():
      index, value = int(token.split(':')[0]), float(token.split(':')[1])
      vector[index] = value
    data.append(vector)
    labels.append(label)
  return torch.tensor(data), torch.tensor(labels)

with open('/content/drive/My Drive/Data_Colab/words_idf.txt', encoding = 'latin1') as f:
  vocab_size = len(f.read().splitlines())
X_train, Y_train = load_data(
    '/content/drive/My Drive/Data_Colab/train_tf_idf_vector.txt', vocab_size)
X_test,Y_test = load_data(
    '/content/drive/My Drive/Data_Colab/test_tf_idf_vector.txt', vocab_size)

- Xây dựng các tham số cho mạng sử dụng torch.nn.Linear gồm các weight và bias cho từng tầng
- Shuffle dữ liệu rồi tách ra thành các batch. Mỗi lần sẽ thực hiện lan truyền tiến, tính lỗi cho toàn bộ phần tử trong batch. Sau đó, cập nhật các tham số của mô hình
- Lan truyền tiến (forward): tầng ẩn sử dụng hàm kích hoạt là sigmoid(x) = 1/(1+exp(-x)), tầng đầu ra sử dụng hàm softmax(xi) = exp(xi)/∑j exp(xj)
- Tính lỗi loss dựa trên đầu ra của lan truyền tiến và nhãn thực tế của ví dụ sử dụng hàm cross_entropy
- loss.backward tính gradient
- Sử dụng torch.optim.Adam để cập nhật tham số dựa trên gradient tính được

In [0]:
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

class NeuronNetwork(torch.nn.Module):
  def __init__(self, vocab_size, hidden_size, num_classes):
    super().__init__()
    self._vocab_size = vocab_size
    self._hidden_size = hidden_size
    self._num_classes = num_classes

  #xay dung cau truc mang
  def build_graph(self):
    self._hidden_layer = torch.nn.Linear(self._vocab_size, self._hidden_size)
    self._output_layer = torch.nn.Linear(self._hidden_size, self._num_classes)

  def forward(self,x):
    x = torch.sigmoid(self._hidden_layer(x))
    x = F.softmax(self._output_layer(x), dim = 1)
    return x

  def fit(self, X_train, Y_train, batch_size, max_epochs=50 ,learning_rate=1e-2, threshold=1e-3):
    self.build_graph()
    dataset = TensorDataset(X_train, Y_train)
    data_loader = DataLoader(dataset, batch_size = batch_size, shuffle = True)
    opt = torch.optim.Adam(params = self.parameters(), lr = learning_rate)
    last_loss = 0
    for epoch in range(max_epochs):
      new_loss = 0
      for data,labels in data_loader:
        self.zero_grad()
        #lan truyen tien, tinh dau ra
        prediced = self.forward(data)
        #xac dinh loi
        loss = F.cross_entropy(prediced, labels)
        new_loss += loss
        #lan truyen nguoc
        loss.backward()
        #cap nhat tham so
        opt.step()
      new_loss = new_loss / len(data_loader)
      print('round: {}, loss: {}'.format(epoch, new_loss))
      if abs(last_loss - new_loss) <= threshold:
        return
      last_loss=new_loss

  def predict(self, X):
    return torch.argmax(self.forward(X), dim = 1)

  def compute_accuracy(self,predicted,expected):
    return (predicted == expected).float().mean()


Huấn luyện mạng neural và tính độ chính xác trên tập huấn luyện với tập thử nghiệm

In [3]:
from time import time
t=time()
NN = NeuronNetwork(
    vocab_size = vocab_size,
    hidden_size = 30,
    num_classes = 20
    )
NN.fit(
    X_train = X_train,
    Y_train = Y_train,
    batch_size = 60,
    max_epochs = 50,
    learning_rate = 1e-2,
    threshold = 1e-3
    )
print('training time:',time()-t,'s')
predicted = NN.predict(X_train)
print('train accuracy:', NN.compute_accuracy(predicted, Y_train))
predicted = NN.predict(X_test)
print('test accuracy:', NN.compute_accuracy(predicted, Y_test))

round: 0, loss: 2.8042428493499756
round: 1, loss: 2.4033281803131104
round: 2, loss: 2.2484819889068604
round: 3, loss: 2.1497304439544678
round: 4, loss: 2.1281681060791016
round: 5, loss: 2.105760097503662
round: 6, loss: 2.090346336364746
round: 7, loss: 2.0862843990325928
round: 8, loss: 2.0843913555145264
round: 9, loss: 2.083629846572876
training time: 14.8424711227417 s
train accuracy: tensor(0.9958)
test accuracy: tensor(0.8343)


Các tham số của mạng sau khi huấn luyện

In [4]:
print('hidden layer weight:', NN._hidden_layer.weight.shape)
print(NN._hidden_layer.weight)
print('hidden layer bias:', NN._hidden_layer.bias.shape)
print(NN._hidden_layer.bias)
print('output layer weight:', NN._output_layer.weight.shape)
print(NN._output_layer.weight)
print('output layer bias:', NN._output_layer.bias.shape)
print(NN._output_layer.bias)

hidden layer weight: torch.Size([30, 14612])
Parameter containing:
tensor([[-0.5120,  0.3229, -0.2952,  ...,  0.0392,  0.5703, -0.0040],
        [-0.5765, -0.4505,  0.3326,  ...,  0.2473, -0.2460, -0.0065],
        [-0.1677, -0.4114,  0.0455,  ...,  0.0970,  0.4613,  0.0048],
        ...,
        [-0.4443, -0.1872, -0.0102,  ..., -0.5484, -0.5428, -0.0010],
        [ 0.4402,  0.0920, -0.2620,  ..., -0.1826, -0.3659,  0.0035],
        [-0.4814,  0.0925, -0.3154,  ..., -0.3219, -0.2417, -0.0070]],
       requires_grad=True)
hidden layer bias: torch.Size([30])
Parameter containing:
tensor([ 3.2821e-01,  2.9508e-02,  1.5489e-01,  4.0406e-02,  3.1571e-01,
         2.4742e-01,  4.0878e-01,  7.5129e-02,  3.3925e-01,  5.0604e-01,
         2.0069e-01,  1.9289e-01,  1.4610e-01,  3.8981e-01,  5.5341e-02,
         1.0784e-01,  2.7814e-01,  3.0716e-01,  2.8899e-01, -4.1495e-02,
         5.1494e-02,  2.5715e-02,  8.0200e-02,  3.5032e-02,  2.8047e-01,
         1.2714e-01,  1.9629e-01,  1.9580e-01, -2

# SVMs

In [11]:
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.svm import SVC

def compute_accuracy(predicted,expected):
	matches=np.equal(predicted,expected)
	accuracy=np.sum(matches)/len(expected)
	return accuracy

train_data = csr_matrix(X_train)
test_data = csr_matrix(X_test)

classifier=SVC(
		C=10,
		kernel='rbf',
		gamma=0.1,
		tol=1e-3,
		verbose=True
    )
classifier.fit(train_data, np.array(Y_train))

predicted = classifier.predict(train_data)
accuracy = compute_accuracy(predicted = predicted, expected = np.array(Y_train))
print('train accuracy:',accuracy)
predicted = classifier.predict(test_data)
accuracy = compute_accuracy(predicted = predicted, expected = np.array(Y_test))
print('test accuracy:',accuracy)

[LibSVM]train accuracy: 0.9967297153968535
test accuracy: 0.8272703133297928
